<a href="https://colab.research.google.com/github/siddhanttripathi13/trainity_projects/blob/master/Call%20Volume%20Trend%20Analysis/Call_volume_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Config
sns.set_style('ticks')
plt.style.use('dark_background')
pd.set_option('display.max_columns', None)

In [ ]:
# Import data
df = pd.read_excel('/content/drive/MyDrive/Trainity Assignments/ABC Call Volume Trend Analysis/data/Call_Volume_Trend_Analysis_Project_9.xlsx')
df.head()

In [ ]:
df.info()

In [ ]:
df_work = df.copy(deep=True)

In [ ]:
df_work.rename(columns={'Queue_Time(Secs)':'Q_Time', 'Duration(hh:mm:ss)':'Duration', 'Call_Seconds (s)':'Call_Seconds',
                        'IVR _Duration':'IVR_Duration','Wrapped _By':'Wrapped_By'}, inplace=True)
df_work.info()

In [ ]:
df_work['Duration'] = pd.to_timedelta(df_work['Duration'].astype(str))
df_work.info()

In [ ]:
df_work.head()

In [ ]:
df_work['IVR_Duration'] = pd.to_timedelta(df_work['IVR_Duration'].astype(str))
df_work.head()

In [ ]:
df_work.columns

In [ ]:
df_work.drop_duplicates(ignore_index=True, inplace=True)
df_work.info()

## Exploratory Data Analysis

### Agent Name

In [ ]:
df_work.Agent_Name.nunique()

In [ ]:
df_work.Agent_Name.value_counts().sort_values(ascending=False)

### Queue Time

In [ ]:
df_work.Q_Time.value_counts()

In [ ]:
sns.histplot(data=df_work, x='Q_Time', kde=True)
plt.xlabel('Queue Time (s)')
plt.show()

In [ ]:
sns.boxplot(data=df_work, x='Q_Time')
plt.xlabel('Queue Time (s)')
plt.show()

In [ ]:
df_work['Day_of_month'] = df_work['Date_&_Time'].dt.day
df_work.groupby('Day_of_month')['Q_Time'].mean().plot(kind='bar')
plt.show()

In [ ]:
df_work['Day_name'] = df_work['Date_&_Time'].dt.day_name()
sns.barplot(data=df_work.groupby('Day_name')[['Q_Time']].mean(), x=df_work.groupby('Day_name')['Q_Time'].mean().index, y=df_work.groupby('Day_name')['Q_Time'].mean().values,
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

### Date and Time

In [ ]:
df_work.Time.unique()

In [ ]:
sns.countplot(data=df_work, x='Time')
plt.xlabel("Time of day")
plt.show()

In [ ]:

plt.figure(figsize=(10,5))
sns.countplot(data=df_work, x='Day_name', order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.xlabel("Day of week")
plt.show()

In [ ]:
df_work['Date_&_Time'].dt.strftime("%B").value_counts()

In [ ]:

plt.figure(figsize=(10,5))
sns.countplot(data=df_work, x='Day_of_month')
plt.xlabel('Day of month')
plt.show()

In [ ]:
df['Date_&_Time'].dt.day.unique()

### Duration

In [ ]:
sns.boxplot(data=df_work['Duration'].dt.total_seconds())
plt.show()

In [ ]:
sns.histplot(data=df_work[df_work['Call_Status']!='abandon']['Duration'].dt.total_seconds())
plt.show()

In [ ]:
df_work['Duration'].dt.total_seconds().value_counts().sort_index(ascending=False)

In [ ]:
df_work[df_work['Duration'].dt.total_seconds()>1000]

In [ ]:
df_work.groupby('Day_of_month')['Call_Seconds'].mean().plot(kind='bar')
plt.show()

In [ ]:
sns.barplot(data=df_work.groupby('Day_name')[['Call_Seconds']].mean(), x=df_work.groupby('Day_name')['Call_Seconds'].mean().index, y=df_work.groupby('Day_name')['Call_Seconds'].mean().values,
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.show()

### Call Status

In [ ]:
sns.countplot(data=df_work, x='Call_Status')
plt.show()

In [ ]:

plt.figure(figsize=(10,5))
sns.countplot(data=df_work, x='Day_of_month',hue='Call_Status')
plt.xticks(np.arange(0,24))
plt.xlabel('Day of month')
plt.show()

In [ ]:
sns.countplot(data=df_work, x='Day_name',hue='Call_Status', order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.xlabel('Day of week')
plt.show()

### Wrapped By

In [ ]:
df_work['Wrapped_By'].value_counts().plot(kind='bar')

In [ ]:
sns.countplot(data=df_work, x='Wrapped_By', hue='Call_Status')
plt.show()

### Ringing

In [ ]:
df_work['Ringing'].value_counts()

### IVR Duration

In [ ]:
df_work['IVR_Duration_seconds'] = df_work['IVR_Duration'].dt.total_seconds()
sns.boxplot(data=df_work, y='IVR_Duration_seconds', x='Call_Status')
plt.show()

In [ ]:
df_work[df_work['IVR_Duration'].dt.total_seconds()>3000]

In [ ]:
df_work.groupby('Day_of_month')['IVR_Duration_seconds'].mean().plot(kind='bar')

In [ ]:
sns.barplot(x=df_work.groupby('Day_name')['IVR_Duration_seconds'].mean().index, y=df_work.groupby('Day_name')['IVR_Duration_seconds'].mean().values,
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

In [ ]:
df_work.isna().sum()/len(df_work)

## Descriptive Analysis

#### Average Call Duration per Time Bucket

In [ ]:
avg_call_duration = df_work[df_work['Call_Seconds']>0].groupby('Time_Bucket')[['Call_Seconds']].mean().reset_index()
plt.figure(figsize=(10,5))
ax = sns.barplot(data=avg_call_duration, x='Time_Bucket', y='Call_Seconds',
            order=['9_10','10_11','11_12','12_13','13_14','14_15','15_16','16_17','17_18','18_19','19_20','20_21'])
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
avg_call_duration = df_work[df_work['Call_Status']=='answered']['Call_Seconds'].mean()
avg_call_duration

#### Call Volume per Time Bucket

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(data=df_work, x='Time_Bucket')
ax.bar_label(ax.containers[0])
plt.show()

#### Daytime Manpower Planning

In [ ]:
df_work['Call_Status'].value_counts()/len(df_work)

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=df_work.groupby('Time_Bucket')['Agent_ID'].nunique().index, y=df_work.groupby('Time_Bucket')['Agent_ID'].nunique().values,
            order=['9_10','10_11','11_12','12_13','13_14','14_15','15_16','16_17','17_18','18_19','19_20','20_21'])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df_work, x='Time_Bucket', hue='Call_Status')
plt.show()

In [ ]:
df_work.groupby('Time_Bucket')['Call_Status'].count()

In [ ]:
df_call_status_pivot = df_work.pivot_table(index='Time_Bucket', columns='Call_Status', values='Ringing', aggfunc='count').reindex(index=['9_10','10_11','11_12','12_13','13_14','14_15','15_16','16_17','17_18','18_19','19_20','20_21'])
df_call_status_pivot['answer_rate'] = df_call_status_pivot['answered']/(df_call_status_pivot['abandon'] + df_call_status_pivot['answered'] + df_call_status_pivot['transfer'])
df_call_status_pivot

In [ ]:
df_agents_time = df_work.groupby('Time_Bucket')[['Agent_ID']].nunique().reindex(index=['9_10','10_11','11_12','12_13','13_14','14_15','15_16','16_17','17_18','18_19','19_20','20_21'])

In [ ]:
manpow_plan = pd.merge(df_call_status_pivot, df_agents_time, left_index=True, right_index=True)
manpow_plan

In [ ]:
manpow_plan.rename(columns={'Agent_ID':'Num_Agents'}, inplace=True)
manpow_plan

In [ ]:
manpow_plan['Extra_Agents_required'] = (np.ceil(0.9*manpow_plan['Num_Agents']/manpow_plan['answer_rate'] - manpow_plan['Num_Agents'])).astype(int)
manpow_plan

Assumptions for manpower planning:
1. Call volumes by time bucket on average remain same.
2. Agent efficiency per time bucket remains same.

In [ ]:
manpow_plot = manpow_plan.reset_index()[['Time_Bucket','Num_Agents','Extra_Agents_required']].melt('Time_Bucket', var_name='cols', value_name='vals')
plt.figure(figsize=(10,5))
ax = sns.barplot(data=manpow_plot, x='Time_Bucket', y='vals', hue='cols')
ax.get_legend().set_title("")
ax.legend(fontsize=7.5)
plt.show()

In [ ]:
manpow_plan['Total_Calls'] = manpow_plan['answered']+manpow_plan['abandon']+manpow_plan['transfer']
manpow_plan['Calls_per_day'] = round(manpow_plan['Total_Calls']/23,2)
manpow_plan['Desired_answer_calls'] = round(0.9*manpow_plan['Calls_per_day'],2)
manpow_plan

In [ ]:
working_hours = 9
break_time = 1.5
actual_working_hours = working_hours - break_time
agent_eff = 0.6
call_time = agent_eff*actual_working_hours
calls_answered_per_working_day = call_time*3600/avg_call_duration
calls_answered_per_hour = calls_answered_per_working_day/call_time

manpow_plan['Desired_Agents'] = (np.ceil(manpow_plan['Desired_answer_calls']/calls_answered_per_hour)).astype(int)

manpow_plan

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(x=manpow_plan.index, y=1-manpow_plan['answer_rate'])
plt.ylabel('Abandon Rate')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.lineplot(x=manpow_plan.index, y=manpow_plan['Desired_Agents'], marker='o')
plt.yticks(range(0, 36,2))
plt.show()

#### Nightime Manpower Planning

In [ ]:
avg_daytime_calls = manpow_plan['Desired_answer_calls'].sum()
avg_nightime_calls = 0.3*avg_daytime_calls

night_manpow_plan = pd.DataFrame(
    {
        'Time_Bucket':['9-10','10-11','11-12','12-1','1-2','2-3','3-4','4-5','5-6','6-7','7-8','8-9'],
        'Distribution_30_calls':[3,3,2,2,1,1,1,1,3,4,4,5]
    }
)

night_manpow_plan

In [ ]:
night_manpow_plan['Total_Distribution'] = night_manpow_plan['Distribution_30_calls']*avg_nightime_calls/30
night_manpow_plan['Desired_Agents'] = (np.ceil(night_manpow_plan['Total_Distribution']/calls_answered_per_hour)).astype(int)
night_manpow_plan

In [ ]:
plt.figure(figsize=(8,5))
sns.lineplot(data=night_manpow_plan, x='Time_Bucket', y='Desired_Agents', marker='o')
plt.yticks(range(0,16,2))
plt.show()